In [ ]:
1

In [ ]:
import sys

sys.path.append("../technique")

In [ ]:
from dotenv import load_dotenv

load_dotenv()
import os

In [ ]:
import pandas as pd

In [ ]:
from openfisca_france import CountryTaxBenefitSystem

In [ ]:
from scenario import StrasbourgSurveyScenario

In [ ]:
from mobilite import build_data, compute_result, set_ajustement_mensuel_num

In [ ]:
df2 = pd.read_csv(
    "/home/thomas/Nextcloud/CodeursEnLiberte/EMS/mobilite/20230612_CTS_AbonnésCorrigé.csv",
    encoding="latin_1",
    sep=";",
)

In [ ]:
df2.shape

In [ ]:
df2

In [ ]:
meta_df = pd.read_excel(
    "/home/thomas/Nextcloud/CodeursEnLiberte/EMS/mobilite/titres_individuels_jointure.xlsx"
)

In [ ]:
df3 = df2.merge(meta_df, on="DERN_LIB_PRODUIT")

In [ ]:
df3["idx"] = df3.index.values
set_ajustement_mensuel_num(df3)

In [ ]:
df4 = df3[df3.exclu.isna()]

In [ ]:
dff = df4
rdf = pd.DataFrame(
    data={
        "pu_fichier": dff.ajustement_mensuel_num,
        "pu_calc": dff.ajustement_mensuel_num * 0,
        "tp": dff.ajustement_mensuel_num * 0,
    }
)

In [ ]:
compens_constant = 0

In [ ]:
data, complement_df = build_data(dff, rdf, 20)

In [ ]:
base = CountryTaxBenefitSystem()
base.load_extension("openfisca_france_local")

In [ ]:
reform = base

In [ ]:
reform_scenario = StrasbourgSurveyScenario(reform, data=data)

In [ ]:
res = compute_result(reform_scenario, complement_df, 0, compens_constant)

In [ ]:
res[0].groupby("sample_id").recettes.sum().describe()

In [ ]:
new_period = "2023-09"

In [ ]:
reform_scenario2 = StrasbourgSurveyScenario(reform, data=data, period=new_period)

In [ ]:
res2 = compute_result(
    reform_scenario2, complement_df, 0, compens_constant, period=new_period
)

In [ ]:
res2[0].groupby("sample_id").recettes.sum().describe()

In [ ]:
1.825270 / 1.834543

In [ ]:
res_pp = pd.DataFrame(
    data={
        "sample_id": res[0].sample_id,
        "prix": res[0].pu_calc,
        "prix_r": res2[0].pu_calc,
        "quantité": 1,
        "ajustement_mensuel_num": complement_df.ajustement_mensuel_num.values,
    }
)
res_pp

In [ ]:
field = "prix"
res_pp[field + "_rs"] = res_pp[field + "_r"].astype("str")
suffix = "_r"
count_field = "quantité"
# count_field = "ajustement_mensuel_num"
matrice = (
    pd.pivot_table(
        res_pp[[field, field + suffix, count_field, "sample_id"]],
        columns=field + suffix,
        index=[field, "sample_id"],
        values=count_field,
        aggfunc=sum,
        fill_value=0,
    )
    .reset_index(field)
    .groupby([field])
    .median()
)

In [ ]:
res_pp

In [ ]:
(
    (res_pp["prix"] != res_pp["prix_r"]) * (res_pp["ajustement_mensuel_num"] == 1)
).sum() / 12 / 20

In [ ]:
matrice

In [ ]:
matrice.columns = pd.Index(
    matrice.columns.values.astype("float64").round(2)
)  # .astype('str'))

In [ ]:
matrice

In [ ]:
862 + 2442 + 56 + 592 + 505

In [ ]:
11525 + 32320